## Lecture List (HTML Editor)

### Development Envrionment

In [1]:
import re
import requests
import pandas as pd
import itertools as it
from bs4 import BeautifulSoup

### KAOS Science

In [128]:
titles=[]
names=[]
    
for i in reversed(range(1, 21)):
    res = requests.get('https://ikaos.org/kaos/video/list.php?sort=&page_no=' + str(i))
    soup = BeautifulSoup(res.content, 'html.parser')
    items = soup.select('p')
    

    for j in items:
        if 'title' in str(j):
            found = re.search('<strong>(.+?)</strong>', str(j)).group(1)
            titles.append(found)
            
        elif 'name' in str(j): 
            found = re.search('<p class="name">(.+?)</p>', str(j)).group(1)
            names.append(found)
        
df=pd.DataFrame([titles, names]).T
df.columns = ['title', 'name']
df    

,title,name
0,물질의 기원 | 2015 여름 제7회 카오스 콘서트 &lt;기원&gt; 2강,김성근
1,"눈으로 보는 분자 한 개의 화학 | 2018 가을 카오스강연 '화학의 미스터리, C...",김유수
2,세상 속의 수數다 | 2018 봄 카오스강연 '모든 것의 수다' 1강,고계원
3,(3) 2012 제 1회 카오스 콘서트 '유리알유희',김민형
4,(2) 2012 제 1회 카오스 콘서트 '유리알유희',김민형
...,...,...
343,"생명의 음표, RNA | 2019 서울대 자연과학대학 공개강연 (4)",김빛내리
344,파동의 세계 - 바이올린에서 중력파까지 | 2019 서울대 자연과학대학 공개강연 (3),최선호
345,데이터 과학과 CSI | 2019 서울대 자연과학대학 공개강연 (2),임채영
346,사랑의 배터리? 전기와 화학의 콜라보 | 2019 서울대 자연과학대학 공개강연 (1),정택동


In [129]:
df.to_csv('kaos_science.csv')

In [128]:
text = ""
    
for i in reversed(range(1, 21)):
    res = requests.get('https://ikaos.org/kaos/video/list.php?sort=&page_no=' + str(i))
    soup = BeautifulSoup(res.content, 'html.parser')
    items = soup.select('p')

    for j in items:
        if 'title' in str(j):
            title_found = re.search('<strong>(.+?)</strong>', str(j)).group(1)
            text += (title_found + " ")
            text += ("(" + name_found + ")" + "\n")
            
        elif 'name' in str(j): 
            name_found = re.search('<p class="name">(.+?)</p>', str(j)).group(1)
            
with open('kaos_science.txt', 'w') as f:
    f.write(text)

In [131]:
line_num = 0
with open('kaos_science.txt', 'r') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 3:
            print(line)

물질의 기원 | 2015 여름 제7회 카오스 콘서트 &lt;기원&gt; 2강 (김성근)
눈으로 보는 분자 한 개의 화학 | 2018 가을 카오스강연 '화학의 미스터리, CheMystery' 6강 (김유수)
세상 속의 수數다 | 2018 봄 카오스강연 '모든 것의 수다' 1강 (고계원)


### Art & Study

In [91]:
titles=[]
names=[]
prices=[]
    
for i in it.chain(reversed(range(101, 107)), reversed(range(201, 204)),
                  reversed(range(301, 307)), reversed(range(401, 406))):
    res = requests.get('https://www.artnstudy.com/n_lecture/LecList.asp?LessonPart=' + str(i))
    soup = BeautifulSoup(res.content, 'html.parser')
    items = soup.select('li')

    for j in items:
        
        if i == 103:
            if 'viewlectitle' in str(j): 
                found = re.search('<li class="viewlectitle">(.+?)</li>', str(j)).group(1)
                titles.append(found)
        
            elif 'viewlecteacher' in str(j):
                found = re.compile('[^ ㄱ-ㅣ가-힣+]').sub('', str(j))
                name_pattern = "([ㄱ-ㅣ가-힣]+[ㄱ-ㅣ가-힣]+[ㄱ-ㅣ가-힣])"
                type_name = re.search(name_pattern, found[-16:-11])
                if type_name != None:
                    names.append(type_name.group())         
                
        elif i != 103:
            if 'viewlectitle' in str(j): 
                found = re.search('<li class="viewlectitle">(.+?)</li>', str(j)).group(1)
                titles.append(found)
                
            elif 'viewlecteacher' in str(j):
                found = re.compile('[^ ㄱ-ㅣ가-힣+]').sub('', str(j))
                names.append(found[1:4])
                pattern = "([0-9]+,+[0-9]+원)"
                type_price = re.search(pattern, str(j))
                if type_price != None:
                    prices.append(type_price.group())
                    
                            
df=pd.DataFrame([titles, names, prices]).T
df.columns = ['title', 'name', 'price']

title_name_price_group = {}
names_str = ""
single_names_str = ""
for i in range(len(df['title'])):
    if i <= (len(df['title']) - 2):
        if df['title'][i] == df['title'][i+1]:
            if df['name'][i] != None and df['name'][i+1] != None:
                names_str += (df['name'][i] + " " + df['name'][i+1] + " ")

        else:
            for j in list(set(names_str.split())):
                single_names_str += j + " "
            if df['title'][i] not in title_name_price_group.keys():
                title_name_price_group[df['title'][i]] = [single_names_str, df['price'][i]]
            single_names_str = ""
            names_str = ""  
            
titles=[]
names=[]
prices=[]

for key, value in title_name_price_group.items():
    titles.append(key)
    names.append(value[0])
    prices.append(value[1])       
    
df=pd.DataFrame([titles, names, prices]).T
df.columns = ['title', 'name', 'price']
df     

,title,name,price
0,칸트 『실천이성비판』 읽기,최성환 박남희 김상현 구연상 이석규,"49,000원"
1,행복의 철학사,김재홍 이정우 서동은 윤구병,"49,000원"
2,악(惡)에 대한 철학적 고찰,김영 박정하 이진경 백상현,"49,000원"
3,스피노자의 『에티카』로 읽는 신과 자연 그리고 인간,김태형 이유경 백상현,"49,000원"
4,레비나스 철학입문 : 『시간과 타자』 읽기,김서영 백상현 유충현,"39,000원"
...,...,...,...
587,기초 희랍어Ⅱ,,None
588,기초 라틴어Ⅰ,,None
589,기초 라틴어Ⅱ,,None
590,한자! 무한한 창조력의 세계 Ⅱ,,None


In [93]:
df.to_csv('artnstudy.csv')

In [133]:
titles=[]
names=[]
prices=[]
    
for i in it.chain(reversed(range(101, 107)), reversed(range(201, 204)),
                  reversed(range(301, 307)), reversed(range(401, 406))):
    res = requests.get('https://www.artnstudy.com/n_lecture/LecList.asp?LessonPart=' + str(i))
    soup = BeautifulSoup(res.content, 'html.parser')
    items = soup.select('li')

    for j in items:
        
        if i == 103:
            if 'viewlectitle' in str(j): 
                found = re.search('<li class="viewlectitle">(.+?)</li>', str(j)).group(1)
                titles.append(found)
        
            elif 'viewlecteacher' in str(j):
                found = re.compile('[^ ㄱ-ㅣ가-힣+]').sub('', str(j))
                name_pattern = "([ㄱ-ㅣ가-힣]+[ㄱ-ㅣ가-힣]+[ㄱ-ㅣ가-힣])"
                type_name = re.search(name_pattern, found[-16:-11])
                if type_name != None:
                    names.append(type_name.group())         
                
        elif i != 103:
            if 'viewlectitle' in str(j): 
                found = re.search('<li class="viewlectitle">(.+?)</li>', str(j)).group(1)
                titles.append(found)
                
            elif 'viewlecteacher' in str(j):
                found = re.compile('[^ ㄱ-ㅣ가-힣+]').sub('', str(j))
                names.append(found[1:4])
                pattern = "([0-9]+,+[0-9]+원)"
                type_price = re.search(pattern, str(j))
                if type_price != None:
                    prices.append(type_price.group())
                    
                            
df=pd.DataFrame([titles, names, prices]).T
df.columns = ['title', 'name', 'price']

title_name_price_group = {}
names_str = ""
single_names_str = ""
for i in range(len(df['title'])):
    if i <= (len(df['title']) - 2):
        if df['title'][i] == df['title'][i+1]:
            if df['name'][i] != None and df['name'][i+1] != None:
                names_str += (df['name'][i] + " " + df['name'][i+1] + " ")

        else:
            for j in list(set(names_str.split())):
                single_names_str += j + " "
            if df['title'][i] not in title_name_price_group.keys():
                title_name_price_group[df['title'][i]] = [single_names_str, df['price'][i]]
            single_names_str = ""
            names_str = ""  
            
text = ""

for key, value in title_name_price_group.items():
    text += key + " " 
    if value[0] == None and value[1] != None:
        text += value[1] + "\n" 
    elif value[0] != None and value[1] == None:
        text += "(" + value[0] + ")" + "\n"
    else: 
        text += "(" + value[0] + ")" + " "
        text += value[1] + "\n"      
    
with open('artnstudy.txt', 'w') as f:
    f.write(text)

In [134]:
line_num = 0
with open('artnstudy.txt', 'r') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 3:
            print(line)

칸트 『실천이성비판』 읽기 (최성환 박남희 김상현 구연상 이석규 ) 49,000원
행복의 철학사 (김재홍 이정우 서동은 윤구병 ) 49,000원
악(惡)에 대한 철학적 고찰 (김영 박정하 이진경 백상현 ) 49,000원


### 한국고등교육재단

In [112]:
titles=[]
names=[]

for i in reversed(range(1, 11)):
    res = requests.get('https://www.kfas.or.kr/bbs/board.php?bo_table=higher_edu_videos&sca=%5BDream+Lecture+%EC%B4%88%EC%B2%AD%ED%8A%B9%EA%B0%95%5D+%EA%B0%95%EC%97%B0+%EB%8B%A4%EC%8B%9C%EB%B3%B4%EA%B8%B0&page=' + str(i))
    soup = BeautifulSoup(res.content, 'html.parser')
    items = soup.select('sup')

    for j in items:
        found = re.search('alt=(.+?)style=', str(j)).group(1)
        found = found.replace('"','')
        if 'l' in found: 
            found = found.split('l')
            titles.append(found[0])
            names.append(found[1])
        elif 'I' in found: 
            found = found.split('I')
            titles.append(found[0])
            names.append(found[1])
        elif '|' in found: 
            found = found.split('|')
            text += found[0]
            text += ("(" +found[1] + ")" + "\n")
        else:
            titles.append(found)
        
df=pd.DataFrame([titles, names]).T
df.columns = ['title', 'name']
df    

,title,name
0,['19 Dream Lecture] 제47회 &quot;더 넓은 세상으로!&quot;,정태용 연세대 국제학대학원 교수
1,[2020년 웨비나 다시보기] 지구환경과 나,유성…
2,[2020년 웨비나 다시보기 - Q&amp;A 풀버전] Mortui Vivos Do...,최인철 서울대 심리…
3,[2020년 웨비나 다시보기] Mortui Vivos Docent – 죽음을 바라보...,최인철 서울대 심리학과 교수
4,[2020년 웨비나 다시보기 - Q&amp;A 풀버전] 코로나 바이러스는 불행 바이...,안덕근 서울대 국제대학원 교수
...,...,...
56,[강연] 협상가는 딜레마를 어떻게 해결할까?,None
57,[Q&amp;A 풀버전] 3cm의 장벽을 넘어 모두의 공간으로,None
58,[강연] 3cm의 장벽을 넘어 모두의 공간으로,None
59,[Q&amp;A 풀버전] 미래도시를 상상하다,None


In [113]:
df.to_csv('kfas.csv')

In [135]:
text = ""

for i in reversed(range(1, 11)):
    res = requests.get('https://www.kfas.or.kr/bbs/board.php?bo_table=higher_edu_videos&sca=%5BDream+Lecture+%EC%B4%88%EC%B2%AD%ED%8A%B9%EA%B0%95%5D+%EA%B0%95%EC%97%B0+%EB%8B%A4%EC%8B%9C%EB%B3%B4%EA%B8%B0&page=' + str(i))
    soup = BeautifulSoup(res.content, 'html.parser')
    items = soup.select('sup')

    for j in items:
        found = re.search('alt=(.+?)style=', str(j)).group(1)
        found = found.replace('"','')
        if 'l' in found: 
            found = found.split('l')
            text += found[0]
            text += ("(" + found[1] + ")" + "\n")
        elif 'I' in found: 
            found = found.split('I')
            text += found[0]
            text += ("(" +found[1] + ")" + "\n")
        elif '|' in found: 
            found = found.split('|')
            text += found[0]
            text += ("(" +found[1] + ")" + "\n")
        else:
            text += (found + "\n")
            
with open('kfas.txt', 'w') as f:
    f.write(text)

In [136]:
line_num = 0
with open('kfas.txt', 'r') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 3:
            print(line)

['19 Dream Lecture] 제47회 &quot;더 넓은 세상으로!&quot; 
[2020년 웨비나 다시보기] 지구환경과 나 ( 정태용 연세대 국제학대학원 교수 )
[2020년 웨비나 다시보기 - Q&amp;A 풀버전] Mortui Vivos Docent – 죽음을 바라보며 삶을… 
